In [ ]:
import numpy as np
import time

In [ ]:
# Constants
EMPTY = 0
BLACK = 1
WHITE = -1
SIZE = 8

DIRECTIONS = [
    (0, 1), (1, 0), (0, -1), (-1, 0),
    (1, 1), (1, -1), (-1, 1), (-1, -1)
]

class Reversi:
    def __init__(self):
        self.board = np.zeros((SIZE, SIZE), dtype=int)
        self.board[3, 3], self.board[4, 4] = WHITE, WHITE
        self.board[3, 4], self.board[4, 3] = BLACK, BLACK
        self.current_player = BLACK  # Black goes first
        self.cache = {}  # Cache for storing board evaluations

    def board_to_tuple(self, board):
        """Converts the board state to a hashable tuple."""
        return tuple(map(tuple, board))

    def is_valid_move(self, board, player, x, y):
        """Check for a valid move."""
        if board[x, y] != EMPTY:
            return False

        valid = False
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            line = []
            while 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == -player:
                line.append((nx, ny))
                nx += dx
                ny += dy
            if 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == player and len(line) > 0:
                valid = True

        return valid

    def get_valid_moves(self, board, player):
        """Get all valid moves for the current player."""
        moves = [(x, y) for x in range(SIZE) for y in range(SIZE) if self.is_valid_move(board, player, x, y)]
        return moves

    def make_move(self, board, player, x, y):
        """Place a move and flip discs."""
        if not self.is_valid_move(board, player, x, y):
            return False

        board[x, y] = player
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            line = []
            while 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == -player:
                line.append((nx, ny))
                nx += dx
                ny += dy
            if 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == player:
                for (fx, fy) in line:
                    board[fx, fy] = player

        return True

    def is_game_over(self, board):
        """Check if the game is over."""
        return len(self.get_valid_moves(board, BLACK)) == 0 and len(self.get_valid_moves(board, WHITE)) == 0

    def display_board(self):
        """Display the current board state."""
        print(np.where(self.board == BLACK, "B", np.where(self.board == WHITE, "W", ".")))

    def get_score(self, board):
        """Get the score for each player."""
        black_score = np.sum(board == BLACK)
        white_score = np.sum(board == WHITE)
        return black_score, white_score

    def evaluate_board(self, board):
        """Heuristic function for evaluating the board state."""
        black_score, white_score = self.get_score(board)
        return black_score - white_score

    def alpha_beta(self, board, depth, alpha, beta, maximizing_player, start_time, time_limit):
        """Alpha-beta pruning algorithm with time limit."""
        if time.time() - start_time > time_limit:
            raise TimeoutError  # Terminate search if time limit is exceeded

        board_tuple = self.board_to_tuple(board)
        if board_tuple in self.cache:
            return self.cache[board_tuple]

        if depth == 0 or self.is_game_over(board):
            eval = self.evaluate_board(board)
            self.cache[board_tuple] = eval
            return eval

        valid_moves = self.get_valid_moves(board, BLACK if maximizing_player else WHITE)

        if maximizing_player:
            max_eval = -np.inf
            for move in valid_moves:
                board_copy = board.copy()
                self.make_move(board_copy, BLACK, *move)
                try:
                    eval = self.alpha_beta(board_copy, depth - 1, alpha, beta, False, start_time, time_limit)
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                except TimeoutError:
                    break  # Stop the search when the time limit is reached
                if beta <= alpha:
                    break  # Beta cut-off
            self.cache[board_tuple] = max_eval
            return max_eval
        else:
            min_eval = np.inf
            for move in valid_moves:
                board_copy = board.copy()
                self.make_move(board_copy, WHITE, *move)
                try:
                    eval = self.alpha_beta(board_copy, depth - 1, alpha, beta, True, start_time, time_limit)
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                except TimeoutError:
                    break  # Stop the search when the time limit is reached
                if beta <= alpha:
                    break  # Alpha cut-off
            self.cache[board_tuple] = min_eval
            return min_eval

    def find_best_move(self, time_limit=5):
        """Find the best move using iterative deepening search with a time limit."""
        start_time = time.time()
        best_move = None
        best_value = -np.inf if self.current_player == BLACK else np.inf
        depth = 1

        while True:
            try:
                # Iterative deepening search: increase depth each iteration
                valid_moves = self.get_valid_moves(self.board, self.current_player)

                for move in valid_moves:
                    board_copy = self.board.copy()
                    self.make_move(board_copy, self.current_player, *move)

                    move_value = self.alpha_beta(board_copy, depth, -np.inf, np.inf, self.current_player == WHITE, start_time, time_limit)
                    if self.current_player == BLACK:
                        if move_value > best_value:
                            best_value = move_value
                            best_move = move
                    else:
                        if move_value < best_value:
                            best_value = move_value
                            best_move = move

                depth += 1  # Increase search depth after each successful iteration
                #print(depth)
            except TimeoutError:
                # Time limit reached, return the best move found so far
                break

        return best_move

# Greedy Bot
class GreedyBot:
    def __init__(self):
        pass

    def get_flipped_discs(self, board, player, x, y):
        """Count the number of discs that would be flipped if the player makes a move at (x, y)."""
        flipped_discs = 0
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            line = []
            while 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == -player:
                line.append((nx, ny))
                nx += dx
                ny += dy
            if 0 <= nx < SIZE and 0 <= ny < SIZE and board[nx, ny] == player and len(line) > 0:
                flipped_discs += len(line)
        return flipped_discs

    def find_best_move(self, board, player):
        """Find the move that maximizes the immediate number of flipped discs."""
        valid_moves = [(x, y) for x in range(SIZE) for y in range(SIZE) if Reversi().is_valid_move(board, player, x, y)]
        if not valid_moves:
            return None

        best_move = None
        max_flipped = -1
        for move in valid_moves:
            flipped_discs = self.get_flipped_discs(board, player, *move)
            if flipped_discs > max_flipped:
                max_flipped = flipped_discs
                best_move = move

        return best_move

# Simulate a game between Reversi bot and Greedy bot
def play_game_greedy():
    game = Reversi()
    greedy_bot = GreedyBot()

    while not game.is_game_over(game.board):
        game.display_board()

        if game.current_player == BLACK:
            print("Reversi Bot's move (Black):")
            best_move = game.find_best_move(time_limit=5)
            if best_move:
                game.make_move(game.board, game.current_player, *best_move)
        else:
            print("Greedy Bot's move (White):")
            best_move = greedy_bot.find_best_move(game.board, WHITE)
            if best_move:
                game.make_move(game.board, WHITE, *best_move)

        game.current_player = -game.current_player  # Switch player

    game.display_board()
    black_score, white_score = game.get_score(game.board)
    print(f"Game Over! Final Score -> Black: {black_score}, White: {white_score}")

def play_game_abs():
    game = Reversi()
    greedy_bot = GreedyBot()

    while not game.is_game_over(game.board):
        game.display_board()

        if game.current_player == BLACK:
            print("Reversi Bot 2's move (Black):")
            best_move = game.find_best_move(time_limit=.001)
            if best_move:
                game.make_move(game.board, game.current_player, *best_move)
        else:
            print("Reversi Bot 6's move (White):")
            best_move = game.find_best_move(time_limit=.07)
            if best_move:
                game.make_move(game.board, game.current_player, *best_move)

        game.current_player = -game.current_player  # Switch player

    game.display_board()
    black_score, white_score = game.get_score(game.board)
    print(f"Game Over! Final Score -> Black: {black_score}, White: {white_score}")



In [ ]:
# Run the game simulation
play_game_abs()
